In [1]:
import imp
import sys
# # sys.path.append("")
# if "/home/ecbm4040/project/random-forests-cuda/" not in sys.path:
#     sys.path.insert(0, "/home/ecbm4040/project/random-forests-cuda/")
# from src.python.cuda_utils import CudaUtils

In [70]:
import pandas as pd
import numpy as np
import time
import pycuda
import pycuda.autoinit
import pycuda.driver as cuda
import pycuda.gpuarray as gpuArray
from pycuda.compiler import SourceModule
from numpy import unravel_index
import pdb

cuda.init()
print("Number of CUDA devices available: ", cuda.Device.count())
my_device = cuda.Device(0)
# cc: compute capability
cc = float('%d.%d' % my_device.compute_capability())
print('Device Name: {}'.format(my_device.name()))
print('Compute Capability: {}'.format(cc))
print('Total Device Memory: {} megabytes'.format(my_device.total_memory()//1024**2))



Number of CUDA devices available:  1
Device Name: Tesla T4
Compute Capability: 7.5
Total Device Memory: 15109 megabytes


In [2]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

def generate_random_data(num_dimensions=3, num_samples=100, num_classes=3, random_state=None):
    # Random weighted amounts of each class
    random_states = list(range(random_state, random_state+3+num_classes*num_dimensions))
    np.random.seed(random_states.pop(0))
    cls_probs = np.array([0] + sorted(np.random.uniform(size=(num_classes - 1))) + [1])
    cls_probs = cls_probs[1:] - cls_probs[:-1]
    cls_num_samples = [int(num_samples*p) for p in cls_probs]
    cls_num_samples[0] += num_samples - sum(cls_num_samples)
    np.random.seed(random_states.pop(0))
    means = np.random.normal(1, 10, size=(num_classes,num_dimensions))
    np.random.seed(random_states.pop(0))
    std = np.random.uniform(1, 10, size=(num_classes,num_dimensions))\
    
    X = np.zeros((num_samples, num_dimensions))
    y = np.zeros((num_samples, 1))
    offset = 0
    for class_i in range(num_classes):
        y[offset:offset+cls_num_samples[class_i]] = class_i
        for dim in range(num_dimensions):
            np.random.seed(random_states.pop(0))
            X[offset:offset+cls_num_samples[class_i], dim] = np.random.normal(means[class_i,dim],
                                                                          std[class_i,dim],
                                                                          size=cls_num_samples[class_i])
        offset += cls_num_samples[class_i]
        
    X, y = shuffle(X, y)
    return X, y


In [71]:

class DecisionTreeCudaUtils():

    def __init__(self):
        """
        Attributes for instance of EncoderDecoder module
        """
        return
    
    def get_source_module(self):
        # kernel code wrapper
        kernelwrapper = """
        // Helps in the calculation of the gina scores
        __global__ void calculate_gina_scores(float* impurity_scores,float* X_train,int* y_train,const int unique_classes,const int l,const int w){
            int Dim = threadIdx.y+blockIdx.y*blockDim.y;
            int Row = threadIdx.x+blockIdx.x*blockDim.x;

            if(Dim < w && Row < l){
                float split_value =X_train[Row * w+ Dim];

                float group1_counts[20] = {0};//Max of 20 dimensions which can be increased
                float group2_counts[20] = {0};
                float length1=0;
                float length2=0;
                float sum1=0;
                float sum2=0;

                for(int i=0;i<l;i++){
                    if(X_train[i* w+ Dim]>=split_value){
                        //Belongs to group 1
                        group1_counts[y_train[i]]++;
                        length1++;
                    }
                    else{
                        //Belongs to group 2
                        group2_counts[y_train[i]]++;
                        length2++;
                    }
                }
                float p1 = length1/(length1+length2);
                float p2 = length2/(length1+length2);

                if(length1 > 0){
                    for(int i=0;i<unique_classes;i++){
                        sum1+=(group1_counts[i]*group1_counts[i])/(length1*length1);
                    }
                }
                if(length2 > 0){
                    for(int i=0;i<unique_classes;i++){
                        sum2+=(group2_counts[i]*group2_counts[i])/(length2*length2);
                    }
                }

                float impurity = p1*sum1+p2*sum2;
                // Write our new pixel value out
                impurity_scores[Row * w + Dim] =impurity;

            }
        }

        //////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //Finds the max value of all the gina impurity scores that we have calculated
        #define BLOCKSIZE 1024
        __global__ void find_best_gina_score(int* index, float* all_gina_scores, const int len){
            //loading segment of data in local memory
            __shared__ float scan_array[2*BLOCKSIZE];
            __shared__ float ii_array[2*BLOCKSIZE];
            unsigned int t =threadIdx.x;
            unsigned int start=2*blockIdx.x*blockDim.x;

            if(start+t <len){
                scan_array[t]=all_gina_scores[start+t];
                ii_array[t]=index[start+t];
            }
            if(start+blockDim.x+t <len){
                scan_array[blockDim.x+t]=all_gina_scores[start+blockDim.x+t];
                ii_array[blockDim.x+t]=index[start+blockDim.x+t];
            }

            for (unsigned int stride = blockDim.x;stride > 0; stride /= 2){
                __syncthreads();
                if (t < stride){
                  
                  if(scan_array[t] < scan_array[t+stride]){
                      scan_array[t]=scan_array[t+stride];
                      ii_array[t]= ii_array[t+stride];
                  }
                }            
            }
            __syncthreads();
            if(threadIdx.x==0){
                index[blockIdx.x] = ii_array[threadIdx.x];
                all_gina_scores[blockIdx.x] =scan_array[threadIdx.x];
            }
        }
        
        //////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //takes the input matrix X and returns the labels for l or r
        __global__ void split_data(float* label,float* X, const int bound,const int dim, const int l,const int w){
            int Row = threadIdx.x+blockIdx.x*blockDim.x;
            if(Row < l){
                if(X[Row*w+dim] <= bound){
                    label[Row]=1;
                }
                else{
                    label[Row]=0;
                }
            }           
        }
        """
        return SourceModule(kernelwrapper)
         
    def calculate_score(self, X_train_b, y_train_b):
        
        
        if not isinstance(X_train_b, np.ndarray):
            raise Exception("X_train_b needs to be np.array")
        if not isinstance(y_train_b, np.ndarray):
            raise Exception("y_train_b needs to be np.array")
            
        y_train_b = y_train_b.astype(np.float32)
        X_train_b = X_train_b.astype(np.float32)
            
        # Implement CUDA function
        
        self.mod = self.get_source_module()

        unique_classes = np.unique(y_train_b)
        #Making categorical data into integers
        for j,label in enumerate(unique_classes):
            y_train_b[y_train_b == label]=j
        
        #Fetch the kernel
        start =cuda.Event()
        end = cuda.Event()

        calculate_scores=self.mod.get_function("calculate_gina_scores")

        #Converting to 32 bit
        X_train_b = X_train_b.astype(np.float32)
        y_train_b =y_train_b.astype(np.int32)

        unique_classes=np.array(len(unique_classes)).astype(np.int32)
        row = np.array(X_train_b.shape[0]).astype(np.int32)
        dim = np.array(X_train_b.shape[1]).astype(np.int32)

        #Grid and block dimensions
        blocksize=32
#         blockDim=(blocksize,X_train_b.shape[1],1)
        blockDim=(blocksize, blocksize, 1)
        gridDim =(X_train_b.shape[0]//blocksize+1, X_train_b.shape[1]//blocksize+1, 1)

        #Memory allocation
        X_train_b_gpu = gpuArray.to_gpu(X_train_b)
        y_train_b_gpu = gpuArray.to_gpu(y_train_b)  
        impurity_scores_gpu = gpuArray.zeros_like(X_train_b_gpu)
        
        #run and time the kernel
        start.record()
        print(blockDim)
        print(gridDim)
        calculate_scores(impurity_scores_gpu,
                         X_train_b_gpu,
                         y_train_b_gpu,
                         unique_classes,
                         row,
                         dim,
                         block=blockDim,
                         grid=gridDim)

        # Wait for the event to complete
        end.record()
        end.synchronize()
        time = start.time_till(end)

        #Fetch the impurity scores
        impurity_scores = impurity_scores_gpu.get()
        return impurity_scores
    
    def choose_best_score(self, all_gina_scores: np.array):
        
        if not isinstance(all_gina_scores, np.ndarray):
            raise Exception("all_gina_scores needs to be np.array")
        all_gina_scores = all_gina_scores.astype(np.float32)
        
        all_gina_scores_flatten = all_gina_scores.flatten()
        
        current_scores = all_gina_scores_flatten
        current_indexes = np.array([i for i in range(all_gina_scores_flatten.shape[0])])
        current_indexes = current_indexes.astype(np.int32)
        
        t_total = 0
        count = 0
        block_size = 8
        while True:
            
            new_current_indexes, new_current_scores, t = self.choose_best_score_recurs(current_scores, current_indexes)
            t_total += t
            count += 1
            
            
            if len(current_indexes) <= block_size:
                #print(f"breaking becuase len(current_indexes) is {len(current_indexes)}")
                break
                
            current_scores, current_indexes = new_current_scores, new_current_indexes
            
            
        max_index=unravel_index(int(current_indexes[0]), all_gina_scores.shape)
        return max_index
    
    

    def choose_best_score_recurs(self, all_gina_scores_flatten, current_indexes):
        #Unravel the matrix
        
        self.mod = self.get_source_module()
        

        #Fetch the kernel 
        # start =cuda.Event()
        # end = cuda.Event()

        find_best_gina_score=self.mod.get_function("find_best_gina_score")
        #setting up the eindex matrix
        index = current_indexes


        #Grid and block dimensions
        block_size = 1024
        blockDim=(block_size,1,1)
        num_blocks = all_gina_scores_flatten.shape[0]//block_size+1
        gridDim =(num_blocks,1,1)
        print(f"num_blocks {num_blocks}")

        #Converting to 32 bit
        row =np.float32(all_gina_scores_flatten.shape[0])
        all_gina_scores_flatten=all_gina_scores_flatten.astype(np.float32)

        #memory allocation
        all_gina_scores_gpu=gpuArray.to_gpu(all_gina_scores_flatten)
        index=index.astype(np.int32)
        index_gpu=gpuArray.to_gpu(index)

        #run and time the kernel
        # start.record()
        find_best_gina_score(index_gpu,all_gina_scores_gpu,row,block=blockDim,grid=gridDim)

        # Wait for the event to complete
        # end.record()
        # end.synchronize()
        # time = start.time_till(end)

        #Fetch the impurity scores
        index=index_gpu.get()
        gina_scores=all_gina_scores_gpu.get()
        
        
        
        pdb.set_trace()
        #max_index=unravel_index()

        return index[:num_blocks], gina_scores[:num_blocks], 0

    def split_data(self, X: np.array, y: np.array, bound: float, dim: float):
        
        if not isinstance(X, np.ndarray):
            raise Exception("X needs to be np.array")
        if not isinstance(y, np.ndarray):
            raise Exception("y needs to be np.array")

        self.mod = self.get_source_module()
        # Implement CUDA function
        #Fetch the kernel
        start =cuda.Event()
        end = cuda.Event()

        split_data=self.mod.get_function("split_data")
        #Grid and block dimensions
        blockDim=(1024,1,1)
        gridDim =(X.shape[0]//1024+1,1,1)
        #Converting to 32 bit
        labels = np.zeros(y.shape).astype(np.float32)
        X_32 = X.astype(np.float32)
        row = np.array([X.shape[0]], dtype=np.int32)
        col = np.array([X.shape[1]], dtype=np.int32)
        bound = np.array([bound], dtype=np.int32)
        dim =np.array([dim], dtype=np.int32)
        #Memory allocation
        X_gpu = gpuArray.to_gpu(X_32) 
        labels_gpu = gpuArray.to_gpu(labels)

        #run and time the kernel
        start.record()
        split_data(labels_gpu,X_gpu,bound,dim,row,col,block=blockDim,grid=gridDim)

        # Wait for the event to complete
        end.record()
        end.synchronize()
        time = start.time_till(end)

        #Fetch the impurity scores
        labels = labels_gpu.get().reshape(-1,)

        #code for splitting the child
#         print(labels==0)
        y_l = y[labels==1]
        y_r = y[labels==0]
#         print(y_l,y_r)
#         print(X[0][:])
#         print(f"X.shape is {X.shape}")
#         print(f"labels.shape is {labels.shape}")
        X_l = X[labels==1,:]
        X_r = X[labels==0,:]
        return (X_l, y_l, X_r, y_r)

In [12]:
import imp
import src.python.random_forest
imp.reload(src.python.random_forest)
from src.python.random_forest import RandomForestFromScratch, DecisionTreeNativePython, DecisionTreeCudaBaise
# rf = RandomForestFromScratch(max_depth=3, n_estimators=1)
# rf.fit(X_train, y_train)


In [49]:
X_all, y_all = generate_random_data(num_dimensions=3, num_samples=100, num_classes=3, random_state=153)
# X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.3)
X_train, y_train = X_all, y_all

In [6]:
ref_gina_scores = DecisionTreeNativePython(max_depth=3).calculate_split_scores(X_train, y_train)

In [36]:

dtu = DecisionTreeCudaUtils()
(ref_X_l, ref_y_l, ref_X_r, ref_y_r) = DecisionTreeNativePython(max_depth=3).split_data(X_train, y_train, 4, 0)
(X_l, y_l, X_r, y_r) = dtu.split_data(X_train, y_train, 4, 0)

print(np.allclose(ref_X_l, X_l))
print(np.allclose(ref_y_l, y_l))
print(np.allclose(ref_X_r, X_r))
print(np.allclose(ref_y_r, y_r))

True
True
True
True


In [56]:
impurity_scores =  dtu.calculate_score.calculate_split_scores(X_train, y_train)
np.allclose(impurity_scores, ref_gina_scores, atol=0.01)
# np.count_nonzero(np.abs(impurity_scores - ref_gina_scores) > 0.001)

True

In [10]:
dtu = DecisionTreeCudaUtils()
print("Reference max index is:" , np.unravel_index(np.argmax(ref_gina_scores), ref_gina_scores.shape))
max_index, max_score = dtu.choose_best_score(ref_gina_scores)
print(max_index, max_score)

Reference max index is: (1585, 4)
num_blocks 74
num_blocks 1
num_blocks 1
1585 4


In [ ]:
# current_scores = data.flatten()
# current_indexes = [i for i in range(len(data.flatten()))]
# block_indexes, current_scores, t = dtu.choose_best_score_recurs(current_scores, current_indexes)

In [72]:
# Testing Cuda RF

dtu = DecisionTreeCudaUtils()
dt_cuda = DecisionTreeCudaBaise(max_depth=4)
dt_cuda.calculate_split_scores = dtu.calculate_score
dt_cuda.choose_best_score = dtu.choose_best_score
dt_cuda.split_data = dtu.split_data

In [ ]:
dt_cuda.fit(X_train, y_train)

(32, 32, 1)
(4, 1, 1)
num_blocks 1
> <ipython-input-71-889a17bcbe65>(262)choose_best_score_recurs()
    260         #max_index=unravel_index()
    261 
--> 262         return index[:num_blocks], gina_scores[:num_blocks], 0
    263 
    264     def split_data(self, X: np.array, y: np.array, bound: float, dim: float):

ipdb> c
num_blocks 1
> <ipython-input-71-889a17bcbe65>(262)choose_best_score_recurs()
    260         #max_index=unravel_index()
    261 
--> 262         return index[:num_blocks], gina_scores[:num_blocks], 0
    263 
    264     def split_data(self, X: np.array, y: np.array, bound: float, dim: float):

ipdb> print(index)         print(gina_scores)
*** SyntaxError: invalid syntax
ipdb> print(index);print(gina_scores)
[182]
[0.7725806]
ipdb> c
(32, 32, 1)
(2, 1, 1)
num_blocks 1
> <ipython-input-71-889a17bcbe65>(262)choose_best_score_recurs()
    260         #max_index=unravel_index()
    261 
--> 262         return index[:num_blocks], gina_scores[:num_blocks], 0
    263 
  

numpy.ndarray